In [64]:
import numpy as np 
from scipy.optimize import minimize

In [65]:
class LinearSVC:
    """Linear support vector machine classifier solved by minimize
    the regularized hinge loss

    Attribute:
    ----------
    C: float, default=1.0
        Regularization parameter
    """
    def __init__(self, C=1.0):
        self.C = C

    @staticmethod
    def _cost_grad(w, X, y, C):
        """Compute the loss of linear SVM
        """
        n = X.shape[0]
        X_train = np.c_[X, np.ones(n)]
        # Hinge loss
        distance = 1 - y * np.dot(X_train, w)
        mask = (distance <= 0)
        distance[mask] = 0
        hinge_loss = np.sum(distance)
        loss = 0.5 * np.dot(w, w) + C * hinge_loss
        y[mask] = 0
        grad = w - C * np.sum(y*X_train.T, axis=1) 
        return loss, grad

    def fit(self, X, y):
        d = X.shape[1]
        w0 = np.zeros(d + 1)
        res = minimize(self._cost_grad, jac=True, x0=w0, 
                    args=(X, y, self.C), method='L-BFGS-B')
        w = res.x
        self.coef_ = w[:-1]
        self.intercept_ = w[-1]

    def predict(self, X):
        y = np.dot(X, self.coef_) + self.intercept_
        positive = y > 0
        negtive = y <= 0
        y[positive] = 1
        y[negtive] = -1
        return y

In [66]:
# Test
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)
y_train[y_train==0] = -1
y_test[y_test==0] = -1

In [67]:
cls1 = LinearSVC(C=100.0)
cls1.fit(X, y)
y_pred = cls1.predict(X_test)
accuracy_score(y_test, y_pred)

0.6433566433566433

In [68]:
from sklearn.svm import LinearSVC as skLinearSVC
cls2 =skLinearSVC
cls1.fit(X, y)
y_pred = cls1.predict(X_test)
accuracy_score(y_test, y_pred)

0.35664335664335667